In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
np.set_printoptions(precision=3)
np.random.seed(0)

In [2]:
df = pd.read_csv('longley.txt', sep=',')

In [3]:
df = df.drop(columns = ['Year'])

In [4]:
D = df.rename(columns = {'Year.1':'Year'})

In [5]:
D = D.values

In [6]:
X = D[:,:6]

In [7]:
print(X)

[[  83.     234.289  235.6    159.     107.608 1947.   ]
 [  88.5    259.426  232.5    145.6    108.632 1948.   ]
 [  88.2    258.054  368.2    161.6    109.773 1949.   ]
 [  89.5    284.599  335.1    165.     110.929 1950.   ]
 [  96.2    328.975  209.9    309.9    112.075 1951.   ]
 [  98.1    346.999  193.2    359.4    113.27  1952.   ]
 [  99.     365.385  187.     354.7    115.094 1953.   ]
 [ 100.     363.112  357.8    335.     116.219 1954.   ]
 [ 101.2    397.469  290.4    304.8    117.388 1955.   ]
 [ 104.6    419.18   282.2    285.7    118.734 1956.   ]
 [ 108.4    442.769  293.6    279.8    120.445 1957.   ]
 [ 110.8    444.546  468.1    263.7    121.95  1958.   ]
 [ 112.6    482.704  381.3    255.2    123.366 1959.   ]
 [ 114.2    502.601  393.1    251.4    125.368 1960.   ]
 [ 115.7    518.173  480.6    257.2    127.852 1961.   ]
 [ 116.9    554.894  400.7    282.7    130.081 1962.   ]]


In [8]:
X = scale(X)

In [9]:
print(X)

[[-1.788 -1.594 -0.925 -1.509 -1.457 -1.627]
 [-1.261 -1.333 -0.96  -1.708 -1.305 -1.41 ]
 [-1.29  -1.347  0.54  -1.47  -1.136 -1.193]
 [-1.166 -1.071  0.174 -1.42  -0.964 -0.976]
 [-0.525 -0.61  -1.209  0.731 -0.794 -0.759]
 [-0.343 -0.423 -1.394  1.465 -0.617 -0.542]
 [-0.257 -0.232 -1.463  1.395 -0.346 -0.325]
 [-0.161 -0.255  0.425  1.103 -0.179 -0.108]
 [-0.046  0.102 -0.32   0.655 -0.005  0.108]
 [ 0.279  0.327 -0.41   0.371  0.195  0.325]
 [ 0.643  0.572 -0.284  0.284  0.449  0.542]
 [ 0.873  0.591  1.644  0.045  0.672  0.759]
 [ 1.045  0.987  0.685 -0.081  0.882  0.976]
 [ 1.198  1.194  0.815 -0.138  1.179  1.193]
 [ 1.342  1.356  1.782 -0.051  1.548  1.41 ]
 [ 1.456  1.737  0.899  0.327  1.879  1.627]]


In [10]:
y = D[:,6]

In [11]:
print(y)

[60.323 61.122 60.171 61.187 63.221 63.639 64.989 63.761 66.019 67.857
 68.169 66.513 68.655 69.564 69.331 70.551]


In [12]:
y = scale(y, with_std=False)

In [13]:
print(y)

[-4.994 -4.195 -5.146 -4.13  -2.096 -1.678 -0.328 -1.556  0.702  2.54
  2.852  1.196  3.338  4.247  4.014  5.234]


In [14]:
n = np.size(X,0)
p = np.size(X,1)

In [15]:
n, p

(16, 6)

In [16]:
beta_0 = np.linalg.pinv(X.transpose().dot(X) + np.identity(p)).dot(X.transpose().dot(y))

In [17]:
print(beta_0)

[ 0.896  1.086 -0.744 -0.197  0.789  1.062]


In [18]:
r = y - X.dot(beta_0)

In [19]:
print(r)

[ 0.233 -0.139 -0.251 -0.274 -0.286 -0.597 -0.041 -0.345  0.413  1.203
  0.569 -0.333  0.09   0.258 -0.065 -0.436]


In [20]:
sigma_sqr_0 = r.transpose().dot(r) / n

In [21]:
print(sigma_sqr_0)

0.19322031712381066


In [22]:
tau_0 = 1.0 / (beta_0 * beta_0)

In [23]:
print(tau_0)

[ 1.246  0.848  1.808 25.867  1.604  0.886]


In [25]:
lambda_0 = p * np.sqrt(sigma_sqr_0)/sum(abs(beta_0))

In [26]:
lambda_0

0.5524867860453111

In [27]:
def D_tau(tau_sqr):
    return np.diag(tau_sqr)

In [28]:
D_tau_0 = D_tau(tau_0)

In [29]:
D_tau_0

array([[ 1.246,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.848,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  1.808,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , 25.867,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.604,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.886]])

In [30]:
def beta(D_tau, sigma_sqr):
    A = X.transpose().dot(X) + np.linalg.pinv(D_tau)
    A_plus = np.linalg.pinv(A)
    loc = A_plus.dot(X.transpose().dot(y))
    scale = sigma_sqr * A_plus
    return np.random.multivariate_normal(loc, scale)

In [56]:
def sigma_sqr(beta, D_tau):
    shape = (n - 1)/2 + p/2
    scale = (y - X.dot(beta)).transpose().dot(y - X.dot(beta))/2 \
        + beta.transpose().dot(np.linalg.pinv(D_tau)).dot(beta)/2
    return 1 / np.random.gamma(shape, scale)

In [57]:
def sample(tau_0, data, numSamples):
    tau_i = tau_0
    beta_i = beta(D_tau(tau_i))

In [58]:
beta(D_tau_0, sigma_sqr_0)

array([ 1.141,  0.373, -0.604,  0.009,  1.089,  1.029])

In [59]:
sigma_sqr(beta_0, D_tau_0)

0.019288185311634117